In [ ]:
## This analysis script is used to create Supp Fig. 8b,d 

In [ ]:
#install required R packages 
suppressMessages(install.packages("BiocManager"))
suppressMessages(BiocManager::install(c('tximport','edgeR','variancePartition','BiocParallel','pheatmap')))

In [ ]:
suppressMessages(library('tximport'))
suppressMessages(library('edgeR'))
suppressMessages(library('variancePartition'))
suppressMessages(library('BiocParallel'))
suppressMessages(library('pheatmap'))

#set working directory

dir.create("rnaseq/")
setwd("rnaseq/")


In [ ]:
#download required h5 matrices in this directory. GEO IDs given here

#GSM7881226 - Jurkat Safe Harbor RNA-seq rep1
#GSM7881227 - Jurkat Safe Harbor RNA-seq rep2
#GSM7881233 - Jurkat sg8 RNA-seq rep1
#GSM7881234 - Jurkat sg8 RNA-seq rep1

In [ ]:
#set up design matrix and transcripts-to-gene dataframe

design_df = read.csv("https://raw.githubusercontent.com/broadinstitute/gro-crispri-ctcf/dev/data/sg8_rnaseq.tsv", sep = "\t")
rownames(design_df) = stringr::str_split_fixed(design_df$file, ".h5",2)[,1]
design_df$sample = rownames(design_df)
design_df$condition = as.factor(design_df$condition)
design_df$condition = relevel(design_df$condition, ref = "Safe-Harbor")
names(design_df) = c("path", "condition", "rep", "exp_day", "sample")

t2g = read.table("transcripts_to_genes.txt", stringsAsFactors = F, header=F, sep = "\t")
colnames(t2g) = c("target_id", "ens_gene", "gene_id")
t2g = t2g[,c("target_id","gene_id")]

design_df

In [ ]:
#load h5 matrices 

source("https://raw.githubusercontent.com/mikelove/tximport/devel/R/helper.R")
files = paste0(rownames(design_df),".h5")
names(files) = rownames(design_df)
txi <- tximport(files, type="kallisto", tx2gene=t2g, geneIdCol = "gene_name", importer=read_kallisto_h5)


In [ ]:
#set up count matrix, filter, and normalize

x = txi$counts
isexpr = rowSums(cpm(x)>0) >= nrow(design_df)
sum(isexpr)
y = DGEList(counts = x[isexpr,])
y = calcNormFactors( y )

In [ ]:
#set up threading parameters

param = SnowParam(6, "SOCK", progressbar=TRUE)

In [ ]:
#run variancePartition model

varPart <- fitExtractVarPartModel( cpm(y), ~ (1|condition) + (1|rep), design_df, BPPARAM=param )
vp <- sortCols( varPart )
plotVarPart( vp )

In [ ]:
#run voom with dream weights model

form <- ~ condition + (1|rep) 
vobjDream = voomWithDreamWeights( cpm(y), form, design_df, BPPARAM=param )


In [ ]:
#run dream model

fitmm = dream( vobjDream, form, design_df, BPPARAM=param )


In [ ]:
#run eBayes model 

fitmm = eBayes(fitmm)

In [ ]:
##Uncomment if you would like to save the fitmm object

#saveRDS(fitmm,"~/fitmm_sg8_rnaseq.rds")
#fitmm = readRDS("~/fitmm_sg8_rnaseq.rds")

In [ ]:
#display top 10 significant results between 10mer and Safe Harbor 

topTable( fitmm, coef='condition10-mer', number = 10)

In [ ]:
#extract significant results between 10mer and Safe Harbor 

var_df_res = as.data.frame(topTable( fitmm, coef='condition10-mer', number = Inf))
var_df_res[which((abs(var_df_res$logFC) > 1) & (abs(var_df_res$z.std) > 3)),]

In [ ]:
#volcano plot

ggplot(data= var_df_res, aes(x= logFC, y= -log10(adj.P.Val))) +
     geom_point(colour= 'grey80', size = 2) +
     geom_point(data= var_df_res[which((abs(var_df_res$logFC) > 1) & (abs(var_df_res$z.std) > 3)),], colour= 'red') +
     geom_vline(xintercept= c(-1, 1), colour= 'blue', linetype= 'dashed') +
     geom_hline(yintercept= 1, colour= 'blue', linetype= 'dashed') +
     #geom_point(data= , colour= 'orange')
     xlab('log2 fold-change') +
     ylab('-log10(adj.P.Val)') +
     theme_classic()

In [ ]:
#heatmap of normalized counts of top DE 100 genes 

options(repr.plot.width=15, repr.plot.height=15)

anncol = data.frame(group = design_df$condition)
rownames(anncol) <- rownames(design_df)
pheatmap(mat = vobjDream$E[rownames(topTable( fitmm, coef='condition10-mer', number = 100)),], 
         cluster_cols = T, annotation_col = anncol,  color = hcl.colors(50, "OrRd", rev = T))